In [1]:
import pandas as pd
import pyodbc
import pgeocode
from datamart import fix_me
from fuzzywuzzy import fuzz
import usaddress

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
username = 'vigrose'
password = 'Gryffindor~10946'
q = "DSN=eprdods; UID={}; PWD={}".format(username, password)
ODS = pyodbc.connect(q)
s = "DSN=PRDDW; UID={}; PWD={}".format(username, password)
AMAEDW = pyodbc.connect(s)

In [34]:
def check_match(thing_1, thing_2):
    '''match check'''
    if thing_1 == thing_2:
        return 1
    elif thing_1 in thing_2 or thing_2 in thing_1:
        return 1
    elif fuzz.ratio(thing_1, thing_2) > 60:
        return 1
    else:
        return 0
    
def clean_address_one(add_1):
    if add_1 == 'None':
        addr_1 = ''
    else:
        addr_1 = add_1 + ' '
    return(addr_1)

def error_handle(parsed_string):
    new_dict = {}
    for thing in parsed_string:
        if thing[1] in new_dict.keys():
            a_list = [new_dict[thing[1]], thing[0]]
            new_dict[thing[1]] = max(a_list, key=len)
        else:
            new_dict[thing[1]] = thing[0]
    return(new_dict)

def check_phone_match(phone_1, phone_2):
    if phone_1 != 'None' and phone_2 != 'None' and phone_1 == phone_2:
            return 1
    else:
        return 0

In [4]:
iqvia_query =  """ 
        SELECT DISTINCT 
        B.BUSINESS_NAME,
        B.DBA_NAME,
        B.PHONE,
        B.PHYSICAL_ADDR_1,
        B.PHYSICAL_ADDR_2,
        B.PHYSICAL_CITY,
        B.PHYSICAL_STATE,
        B.PHYSICAL_ZIP,
        P.ME,
        T.AFFIL_TYPE_DESC,
        A.AFFIL_IND,
        A.AFFIL_RANK
        FROM 
        ODS.ODS_IMS_BUSINESS B, ODS.SAS_ODS_IMS_PROVIDER_AFFIL A, ODS.ODS_IMS_PROFESSIONAL P, ODS.ODS_IMS_AFFILIATION_TYPE T
        WHERE  
        B.IMS_ORG_ID = A.IMS_ORG_ID
        AND
        A.PROFESSIONAL_ID = P.PROFESSIONAL_ID
        AND
        A.AFFIL_TYPE_ID = T.AFFIL_TYPE_ID
        AND
        P.CURRENT_BATCH_FLAG='Y'
        AND
        A.CURRENT_BATCH_FLAG='Y'
        AND
        B.CURRENT_BATCH_FLAG='Y'
        """

In [8]:
symphony_query = \
        """
        SELECT DISTINCT
        d.ADDR_LINE_2_TXT AS ADDR_2,
        d.ADDR_LINE_1_TXT AS ADDR_1,
        d.ADDR_CITY_NAM AS CITY,
        d.ADDR_ST_CDE AS STATE,
        d.ADDR_ZIP_CDE AS ZIPCODE,
        d.ADDR_FRST_TLPHN_NBR AS PHONE,
        l.OTHER_ID AS SYM_ME
        FROM
        ODS.PRACTITIONER_DEMOGRAPHIC_LAYOUT d, ODS.PRACTITIONER_ADDL_IDS_LAYOUT l
        WHERE
        d.DS_PRCTR_ID = l.DS_PRCTR_ID
        AND
        l.ID_QLFR_TYP_CDE = 38
        """ 

In [9]:
# iqvia = pd.read_sql(con=ODS, sql=iqvia_query)
# print(iqvia.head())
symphony = pd.read_sql(con=ODS, sql=symphony_query)
symphony.head()

,ADDR_2,ADDR_1,CITY,STATE,ZIPCODE,PHONE,SYM_ME
0,None,4899 REYNOLDS LN,BIRMINGHAM,AL,35242,(727) 642-5087,0010200001
1,None,171 ASHLEY AVE,CHARLESTON,SC,29425,(843) 792-3537,0010200002
2,None,500 CENTRE PARK DR,ASHEVILLE,NC,28805,(828) 254-4337,0010200003
3,None,14502 W MEEKER BLVD,SUN CITY WEST,AZ,85375,None,0010200004
4,STE 440,4429 CLARA ST,NEW ORLEANS,LA,70115,(504) 842-9618,0010200005


In [69]:
# polos = pd.read_csv('../../Data/POLO_Filter/Filtered_POLOs_Older.csv')
dhc = pd.read_csv('../../Data/DHC/DHC_2021-04.csv')

In [39]:
polos['ME']=fix_me(polos.ME)

In [68]:
dhc.columns

AttributeError: 'str' object has no attribute 'columns'

In [14]:
dhc.columns = [c.replace(' ','_') for c in dhc.columns.values]

In [20]:
dhc_merge = pd.merge(polos, dhc, on='ME').drop_duplicates()

In [19]:
polos.drop_duplicates()

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POST_KEY,POST_CD_ID_PPMA,ADDR_1_PPMA,ADDR_2_PPMA,ADDR_3_PPMA,CITY_PPMA,STATE_PPMA,ZIP_PPMA,FROM_DT_PPMA,POLO_NEWER
0,102000412,A,1,2.0,ERIC J BRAY MD,BRAY,ERIC,JAY,None,None,...,10364428,31635528,108 Sullivan Rd,NaN,NaN,Muscle Shoals,AL,35661,2020-09-11 21:12:13.053860,False
1,102011295,A,1,2.0,KARIN M HARDIMAN MD,HARDIMAN,KARIN,MARIE,None,None,...,15930988,26917161,3732 Dunbarton Dr,NaN,NaN,Mountain Brk,AL,35223,2019-07-02 21:12:11.402334,False
2,102011741,A,1,2.0,ERIK P HESS MD,HESS,ERIK,PAUL,None,None,...,726067,31643069,3423B Stokesmont Rd,NaN,NaN,Nashville,TN,37215,2020-09-11 21:12:13.367831,False
3,102020502,A,1,2.0,THEOLYN N PRICE MD,PRICE,THEOLYN,NAN,None,None,...,15868653,28742576,210 Sherwood Pass,NaN,NaN,Roswell,GA,30075,2018-07-16 20:43:55.299512,False
4,102021126,A,1,2.0,WENDY D KRIEGEL MD,KRIEGEL,WENDY,D WASDEN,None,None,...,18210829,25948619,566 Lake Cameron Dr,NaN,NaN,Pike Road,AL,36064,2015-08-08 17:58:47.014798,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23189,95708880109,A,1,2.0,IGOR LOVRINCEVIC MD,LOVRINCEVIC,IGOR,None,None,None,...,6617837,29234404,1315 N Orange Grove Ave,NaN,NaN,W Hollywood,CA,90046,2018-11-07 20:48:28.700113,False
23190,95708880133,A,1,1.0,DJENITA BUTULIJA MD,BUTULIJA,DJENITA,None,None,None,...,9903490,24083707,2002 Holcombe Blvd,NaN,NaN,Houston,TX,77030,2018-03-13 20:48:07.961617,False
23191,95708930033,A,1,2.0,VEDRAN USCHUPLICH MD,USCHUPLICH,VEDRAN,None,None,None,...,2606430,30500981,224 Autumn Cir,NaN,NaN,Columbia,SC,29206,2020-03-03 21:10:01.807952,False
23192,96501860058,A,1,2.0,MWENDALUBI YASA MD,YASA,MWENDALUBI,None,None,None,...,8293896,31398465,2651 Moolio Pl,NaN,NaN,Kihei,HI,96753,2020-07-29 21:11:43.525938,False


In [22]:
dhc_merge = dhc_merge.fillna('None')
dhc_merge[dhc_merge.Phone_Number!='None']

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,State,City,Address,First_Name,Physician_Name,Middle_Name,Primary_Specialty,Primary_Hospital_Affiliation,Last_Name,Address1
0,102011295,A,1,2.0,KARIN M HARDIMAN MD,HARDIMAN,KARIN,MARIE,None,None,...,AL,Birmingham,2000 6th Ave N,Karin,Karin Hardiman,Marie,Gastroenterology,"Birmingham VA Medical Center (Birmingham, AL)",Hardiman,None
1,102011741,A,1,2.0,ERIK P HESS MD,HESS,ERIK,PAUL,None,None,...,AL,Birmingham,1802 6th Ave N,Erik,Erik Hess,P,Emergency Medicine,Vanderbilt University Medical Center (Nashvill...,Hess,None
2,102020502,A,1,2.0,THEOLYN N PRICE MD,PRICE,THEOLYN,NAN,None,None,...,GA,Roswell,2500 Hospital Blvd,Theolyn,Theolyn Price,Nan,Surgery - Thoracic Surgery,"WellStar Kennestone Hospital (Marietta, GA)",Price,None
3,102021126,A,1,2.0,WENDY D KRIEGEL MD,KRIEGEL,WENDY,D WASDEN,None,None,...,AL,Montgomery,215 Perry Hill Rd,Wendy,Wendy Kriegel,W,Radiology - Diagnostic Radiology,"Baptist Medical Center East (Montgomery, AL)",Kriegel,None
6,102031164,A,1,1.0,EMILY Y MELTON MD,MELTON,EMILY,YVETTE,None,BAPTIST MEMORIAL HOSPITAL- DESOTO,...,AL,Birmingham,832 Princeton Ave Sw,Emily,Emily Bell,Melton,Obstetrics/Gynecology,"Princeton Baptist Medical Center (Birmingham, AL)",Bell,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20350,95708880061,A,1,2.0,SASA STRUNJAS MD,STRUNJAS,SASA,None,None,None,...,KY,Bowling Green,350 Park St,Sasa,Sasa Strunjas,None,Psychiatry,"Lexington Medical Center (West Columbia, SC)",Strunjas,#204
20351,95708880109,A,1,2.0,IGOR LOVRINCEVIC MD,LOVRINCEVIC,IGOR,None,None,None,...,CA,Monterey Park,525 N Garfield Ave,Igor,Igor Lovrincevic,None,Anesthesiology - Anesthesiologist,"Garfield Medical Center (Monterey Park, CA)",Lovrincevic,None
20353,95708880133,A,1,1.0,DJENITA BUTULIJA MD,BUTULIJA,DJENITA,None,None,None,...,TX,Houston,2002 Holcombe Blvd,Djenita,Djenita Butulija,None,Family Practice,"Michael E DeBakey VA Medical Center (Houston, TX)",Butulija,None
20354,95708930033,A,1,2.0,VEDRAN USCHUPLICH MD,USCHUPLICH,VEDRAN,None,None,None,...,DE,Wilmington,1601 Kirkwood Hwy,Vedran,Vedran Uschuplich,None,Pathology,Columbia VA Health Care System (AKA Jennings B...,Uschuplich,None


In [25]:
dhc_merge[['TELEPHONE_NUMBER', 'Phone_Number']]

,TELEPHONE_NUMBER,Phone_Number
0,7349365738.0,205.975.3000
1,5072842511.0,800.822.8816
2,None,770.424.9732
3,2365341328.0,334.272.4670
4,5082893480.0,None
...,...,...
20350,8007377900.0,270.843.5103
20351,7168617784.0,716.861.7784
20353,None,713.791.1414
20354,None,302.994.2511


In [35]:
polo_match_count = 0
ppma_match_count = 0
phone_match_count = 0
for row in dhc_merge.itertuples():
    polo = row.ADDR_1_POLO
    dhc = row.Address
    ppma = row.MAILING_LINE_2
    dhc_phone = str(row.Phone_Number).replace('.','')
    phone = str(row.TELEPHONE_NUMBER).replace('.0','')
    polo_match_count += check_match(polo, dhc)
    ppma_match_count += check_match(dhc.upper(),ppma)
    phone_match_count += check_phone_match(dhc_phone, phone)

In [38]:
phone_match_count

3050

In [41]:
polos['SYM_ME'] = [x[:-1] for x in polos.ME]

In [44]:
symphony_merge = pd.merge(polos, symphony, on = 'SYM_ME')

In [53]:
symphony_merge = symphony_merge.fillna('None')
symphony_merge[symphony_merge.PHONE!='None']

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,ZIP_PPMA,FROM_DT_PPMA,POLO_NEWER,SYM_ME,ADDR_2,ADDR_1,CITY_y,STATE_y,ZIPCODE,PHONE
0,00102000412,A,1,2.0,ERIC J BRAY MD,BRAY,ERIC,JAY,None,None,...,35661,2020-09-11 21:12:13.053860,False,0010200041,STE 202,110 29TH AVE N,NASHVILLE,TN,37203,(615) 327-4304
1,00102011295,A,1,2.0,KARIN M HARDIMAN MD,HARDIMAN,KARIN,MARIE,None,None,...,35223,2019-07-02 21:12:11.402334,False,0010201129,2ND FL,1500 E MEDICAL CENTER DR,ANN ARBOR,MI,48109,(734) 936-5738
2,00102011741,A,1,2.0,ERIK P HESS MD,HESS,ERIK,PAUL,None,None,...,37215,2020-09-11 21:12:13.367831,False,0010201174,None,619 19TH ST S,BIRMINGHAM,AL,35249,(205) 975-9358
3,00102020502,A,1,2.0,THEOLYN N PRICE MD,PRICE,THEOLYN,NAN,None,None,...,30075,2018-07-16 20:43:55.299512,False,0010202050,STE 4120,61 WHITCHER ST NE,MARIETTA,GA,30060,(770) 424-9732
4,00102021126,A,1,2.0,WENDY D KRIEGEL MD,KRIEGEL,WENDY,D WASDEN,None,None,...,36064,2015-08-08 17:58:47.014798,False,0010202112,None,215 PERRY HILL RD,MONTGOMERY,AL,36109,(334) 272-4670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23073,95708880061,A,1,2.0,SASA STRUNJAS MD,STRUNJAS,SASA,None,None,None,...,29205,2020-09-11 21:12:14.554177,False,9570888006,STE 204,350 PARK ST,BOWLING GREEN,KY,42101,(270) 843-5103
23074,95708880079,A,1,2.0,SIMA SULER MD,SULER,SIMA,None,None,None,...,33981,2017-08-09 20:40:29.181763,False,9570888007,None,1405 SHADY AVE,PITTSBURGH,PA,15217,(412) 420-2400
23076,95708880133,A,1,1.0,DJENITA BUTULIJA MD,BUTULIJA,DJENITA,None,None,None,...,77030,2018-03-13 20:48:07.961617,False,9570888013,None,2002 HOLCOMBE BLVD,HOUSTON,TX,77030,(713) 791-1414
23077,95708930033,A,1,2.0,VEDRAN USCHUPLICH MD,USCHUPLICH,VEDRAN,None,None,None,...,29206,2020-03-03 21:10:01.807952,False,9570893003,None,1601 KIRKWOOD HWY,WILMINGTON,DE,19805,(302) 994-2511


In [50]:
symphony_merge['']

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,STATE_PPMA,ZIP_PPMA,FROM_DT_PPMA,POLO_NEWER,SYM_ME,ADDR_2,ADDR_1,CITY_y,STATE_y,ZIPCODE
PHONE,,,,,,,,,,,,,,,,,,,,,
(507) 284-2511,53,53,53,53,53,53,53,53,53,53,...,53,53,53,53,53,6,53,53,53,53
(952) 595-1100,27,27,27,27,27,27,27,27,27,27,...,27,27,27,27,27,27,27,27,27,27
(254) 724-2111,26,26,26,26,26,26,26,26,26,26,...,26,26,26,26,26,5,26,26,26,26
(904) 953-2000,25,25,25,25,25,25,25,25,25,25,...,25,25,25,25,25,0,25,25,25,25
(303) 338-4545,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,3,19,19,19,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
(419) 468-4841,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,1,1,1
(419) 471-4491,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,1,1,1
(419) 483-4040,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,1,1,1


In [57]:
polo_match_count = 0
ppma_match_count = 0
phone_match_count = 0
for row in symphony_merge.itertuples():
    polo = str(row.ADDR_1_POLO).upper()
    symphony = row.ADDR_1
    ppma = row.MAILING_LINE_2
    symphony_phone = str(row.PHONE).replace('-','').replace(')','').replace('(','').replace(' ','')
    phone = str(row.TELEPHONE_NUMBER).replace('.0','')
    polo_match_count += check_match(polo, symphony)
    ppma_match_count += check_match(symphony, ppma)
    phone_match_count += check_phone_match(symphony_phone, phone)
print(f'{polo_match_count} match POLO')
print(f'{phone_match_count} match phone')
print(f'{ppma_match_count} match PPMA')

9473 match POLO
3761 match phone
1348 match PPMA


In [55]:
polo_match_count

44

In [58]:
iqvia.head()

,BUSINESS_NAME,DBA_NAME,PHONE,PHYSICAL_ADDR_1,PHYSICAL_ADDR_2,PHYSICAL_CITY,PHYSICAL_STATE,PHYSICAL_ZIP,ME,AFFIL_TYPE_DESC,AFFIL_IND,AFFIL_RANK
0,SOUTH BALDWIN REGIONAL MEDICAL CENTER,None,2519493400,1613 N MCKENZIE ST,None,FOLEY,AL,36535-2247,0010200001,Admitting,A,6
1,LARGO MEDICAL CENTER,None,7275885200,201 14TH ST SW,None,LARGO,FL,33770-3133,0010200001,Admitting,A,6
2,"MISSION HEALTH SYSTEM, INC",MISSION HOSPITAL SAINT JOSEPH CAMPUS,8282131111,428 BILTMORE AVE,None,ASHEVILLE,NC,28801-4502,0010200003,Admitting,A,6
3,MISSION HEALTH,MISSION HOSPITAL MEMORIAL CAMPUS,8282131111,509 BILTMORE AVE,None,ASHEVILLE,NC,28801-4601,0010200003,Admitting,A,6
4,BANNER DEL E WEBB MEDICAL CENTER,None,6235244000,14502 W MEEKER BLVD,None,SUN CITY WEST,AZ,85375-5282,0010200004,Attending,P,10


In [61]:
iqvia_merge = pd.merge(iqvia, polos, left_on='ME', right_on='SYM_ME')
iqvia_merge = iqvia_merge.fillna('None')

In [62]:
polo_match_count = 0
ppma_match_count = 0
phone_match_count = 0
for row in iqvia_merge.itertuples():
    polo = str(row.ADDR_1_POLO).upper()
    iqvia = row.PHYSICAL_ADDR_1
    ppma = row.MAILING_LINE_2
    iqvia_phone = str(row.PHONE)
    phone = str(row.TELEPHONE_NUMBER).replace('.0','')
    polo_match_count += check_match(polo, symphony)
    ppma_match_count += check_match(iqvia, ppma)
    phone_match_count += check_phone_match(iqvia_phone, phone)
print(f'{polo_match_count} match POLO')
print(f'{phone_match_count} match phone')
print(f'{ppma_match_count} match PPMA')

324 match POLO
0 match phone
957 match PPMA


In [63]:
iqvia_merge[['TELEPHONE_NUMBER', 'PHONE']]

,TELEPHONE_NUMBER,PHONE
0,2563400012.0,2563864196
1,2563400012.0,2567372000
2,7349365738.0,7349364000
3,5072842511.0,2059343411
4,None,7707512500
...,...,...
26489,2159557952.0,4349240000
26490,7168617784.0,6265732222
26491,None,7137911414
26492,None,8037764000


In [65]:
iqvia_merge[iqvia_merge.PHONE!='None'].drop_duplicates('ME_y')

,BUSINESS_NAME,DBA_NAME,PHONE,PHYSICAL_ADDR_1,PHYSICAL_ADDR_2,PHYSICAL_CITY,PHYSICAL_STATE,PHYSICAL_ZIP,ME_x,AFFIL_TYPE_DESC,...,POST_CD_ID_PPMA,ADDR_1_PPMA,ADDR_2_PPMA,ADDR_3_PPMA,CITY_PPMA,STATE_PPMA,ZIP_PPMA,FROM_DT_PPMA,POLO_NEWER,SYM_ME
0,HELEN KELLER HOSPITAL,None,2563864196,1300 S MONTGOMERY AVE,None,SHEFFIELD,AL,35660-6334,0010200041,Admitting,...,31635528,108 Sullivan Rd,None,None,Muscle Shoals,AL,35661,2020-09-11 21:12:13.053860,False,0010200041
2,UNIVERSITY OF MICHIGAN CS MOTT CHILDRENS HOSPITAL,None,7349364000,1540 E HOSPITAL DR,None,ANN ARBOR,MI,48109-4000,0010201129,Admitting,...,26917161,3732 Dunbarton Dr,None,None,Mountain Brk,AL,35223,2019-07-02 21:12:11.402334,False,0010201129
3,UAB HOSPITAL BIRMINGHAM,None,2059343411,1802 6TH AVE S,None,BIRMINGHAM,AL,35233-1932,0010201174,IDN Affiliated,...,31643069,3423B Stokesmont Rd,None,None,Nashville,TN,37215,2020-09-11 21:12:13.367831,False,0010201174
4,WELLSTAR NORTH FULTON HOSPITAL,None,7707512500,3000 HOSPITAL BLVD,None,ROSWELL,GA,30076-4915,0010202050,IDN Affiliated (Inferred),...,28742576,210 Sherwood Pass,None,None,Roswell,GA,30075,2018-07-16 20:43:55.299512,False,0010202050
6,CENTRAL ALABAMA VETERANS HEALTH CARE SYSTEM WE...,None,3342724670,215 PERRY HILL RD,None,MONTGOMERY,AL,36109-3725,0010202112,Attending,...,25948619,566 Lake Cameron Dr,None,None,Pike Road,AL,36064,2015-08-08 17:58:47.014798,False,0010202112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26487,JEFFERSON HOSPITAL FOR NEUROSCIENCE,None,2155031000,900 WALNUT ST,None,PHILADELPHIA,PA,19107-5191,9570886006,Admitting,...,30907026,PO Box 800394,1221 Lee St,Fl 4,Charlottesvle,VA,22908,2021-04-27 20:47:51.278694,False,9570886006
26490,GARFIELD MEDICAL CENTER,"AHMC GARFIELD MEDICAL CENTER, LP",6265732222,525 N GARFIELD AVE,None,MONTEREY PARK,CA,91754-1202,9570888010,Admitting,...,29234404,1315 N Orange Grove Ave,None,None,W Hollywood,CA,90046,2018-11-07 20:48:28.700113,False,9570888010
26491,MICHAEL E DEBAKEY VA MEDICAL CENTER,None,7137911414,2002 HOLCOMBE BLVD,None,HOUSTON,TX,77030-4211,9570888013,Attending,...,24083707,2002 Holcombe Blvd,None,None,Houston,TX,77030,2018-03-13 20:48:07.961617,False,9570888013
26492,WILLIAM JENNINGS BRYAN DORN DEPARTMENT OF VETE...,DORN VA MEDICAL CENTER,8037764000,6439 GARNERS FERRY RD,None,COLUMBIA,SC,29209-1638,9570893003,Admitting,...,30500981,224 Autumn Cir,None,None,Columbia,SC,29206,2020-03-03 21:10:01.807952,False,9570893003


In [95]:
dhc_merge['ME']=fix_me(dhc_merge.ME)

In [116]:
# symphony_merge = pd.merge(polos, symphony, on = 'SYM_ME', how='left', suffixes = ['','_symphony'])
# merge_2 = pd.merge(symphony_merge, dhc_merge, on='ME', how='outer', suffixes = ['','_dhc'])
merge_3 = pd.merge(merge_2, iqvia_merge, on='ME', how='outer', suffixes = ['','_iqvia'])

In [87]:
symphony_merge = symphony_merge.rename(columns={'CITY_x':'CITY', 'STATE_x': 'STATE'})

In [77]:
dhc_merge[dhc_merge.ME.isin(symphony_merge.ME)==False]

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,State,City,Address,First_Name,Physician_Name,Middle_Name,Primary_Specialty,Primary_Hospital_Affiliation,Last_Name,Address1
56,102140125,A,1,2.0,JENNY LEE MD,LEE,JENNY,None,None,None,...,CA,Sacramento,6600 Bruceville Rd,Jenny,Jenny Lee,None,Pediatric Medicine,Kaiser Permanente - South Sacramento Medical C...,Lee,None
262,375060825,A,1,2.0,MARY E MILLER DO,MILLER,MARY,ELIZABETH,None,None,...,MI,Saginaw,1500 Weiss St,Mary,Mary Miller,Elizabeth,Physical Medicine and Rehabilitation,"Marion VA Medical Center (Marion, IL)",Miller,None
783,512031501,A,1,2.0,DAVID S KIM MD,KIM,DAVID,SHINYOUNG,None,None,...,CA,Loma Linda,11201 Benton St,David,David Kim,Shin Young,Anesthesiology - Anesthesiologist,VA Loma Linda Healthcare System (AKA Jerry L P...,Kim,None
1295,576081537,A,1,2.0,BRYON D THOMSON DO,THOMSON,BRYON,DANIEL,None,None,...,CA,Moreno Valley,26520 Cactus Ave,Bryon,Bryon Thomson,Daniel,Radiology - Diagnostic Radiology,Riverside University Health System Medical Cen...,Thomson,None
1337,576141173,A,1,2.0,KARTHIKEYAN VISWANATHAN DO,VISWANATHAN,KARTHIKEYAN,None,None,None,...,AR,Conway,1500 N Museum Rd,Karthikeyan,Karthikeyan Viswanathan,None,Internal Medicine,Shiprock - Northern Navajo Medical Center (Shi...,Viswanathan,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18961,67209130049,A,1,2.0,PRIYADARSHANI SHARMA MD,SHARMA,PRIYADARSHANI,None,None,None,...,ME,Bangor,489 State St,Priyadarshani,Priyadarshani Sharma,None,Hospitalist,Northern Light Eastern Maine Medical Center (B...,Sharma,None
19305,70421030250,A,1,2.0,RABIYA A KHAN MD,KHAN,RABIYA,AKRAM,None,None,...,TX,Houston,7200 Cambridge St,Rabiya,Rabiya Khan,None,Internal Medicine,UF Health Leesburg Hospital (FKA Leesburg Regi...,Khan,"10th Floor, Suite 10C"
19324,70421970261,A,1,2.0,MUHAMMAD Z RIZWAN MD,RIZWAN,MUHAMMAD,ZAHID,None,None,...,MD,Hagerstown,11116 Medical Campus Rd,Muhammad,Muhammad Rizwan,None,Critical Care (Intensivists),Riverside Regional Medical Center (Newport New...,Rizwan,None
19335,70424100011,A,1,2.0,NONE MARIUM MD,MARIUM,NONE,None,None,DC043 00,...,MO,Columbia,1 Hospital Dr,Unknown,Unknown Marium,None,Hospitalist,"University Hospital (Columbia, MO)",Marium,None


In [85]:
symphony_merge.columns

Index(['ME', 'RECORD_ID', 'UPDATE_TYPE', 'ADDRESS_TYPE', 'MAILING_NAME',
       'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'MAILING_LINE_1',
       ...
       'ZIP_PPMA', 'FROM_DT_PPMA', 'POLO_NEWER', 'SYM_ME', 'ADDR_2', 'ADDR_1',
       'CITY_y', 'STATE_y', 'ZIPCODE', 'PHONE'],
      dtype='object', length=107)

In [90]:
dhc_merge['SYM_ME'] = [str(x)[:-1] for x in dhc_merge.ME]

In [99]:
iqvia_merge.columns

Index(['BUSINESS_NAME', 'DBA_NAME', 'PHONE', 'PHYSICAL_ADDR_1',
       'PHYSICAL_ADDR_2', 'PHYSICAL_CITY', 'PHYSICAL_STATE', 'PHYSICAL_ZIP',
       'ME_x', 'AFFIL_TYPE_DESC',
       ...
       'POST_CD_ID_PPMA', 'ADDR_1_PPMA', 'ADDR_2_PPMA', 'ADDR_3_PPMA',
       'CITY_PPMA', 'STATE_PPMA', 'ZIP_PPMA', 'FROM_DT_PPMA', 'POLO_NEWER',
       'SYM_ME'],
      dtype='object', length=113)

In [115]:
iqvia_merge = iqvia_merge.rename(columns={'ME':'ME_iq','ME_y':'ME'})

In [104]:
iqvia_merge['ME_x']

0        0010200041
1        0010200041
2        0010201129
3        0010201174
4        0010202050
            ...    
26489    9570886006
26490    9570888010
26491    9570888013
26492    9570893003
26493    9650186005
Name: ME_x, Length: 26494, dtype: object

In [107]:
iqvia_merge.columns

Index(['BUSINESS_NAME', 'DBA_NAME', 'PHONE', 'PHYSICAL_ADDR_1',
       'PHYSICAL_ADDR_2', 'PHYSICAL_CITY', 'PHYSICAL_STATE', 'PHYSICAL_ZIP',
       'ME_x', 'AFFIL_TYPE_DESC',
       ...
       'POST_CD_ID_PPMA', 'ADDR_1_PPMA', 'ADDR_2_PPMA', 'ADDR_3_PPMA',
       'CITY_PPMA', 'STATE_PPMA', 'ZIP_PPMA', 'FROM_DT_PPMA', 'POLO_NEWER',
       'SYM_ME'],
      dtype='object', length=113)

In [108]:
merge_2.columns

Index(['ME', 'RECORD_ID', 'UPDATE_TYPE', 'ADDRESS_TYPE', 'MAILING_NAME',
       'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'MAILING_LINE_1',
       ...
       'City', 'Address', 'First_Name', 'Physician_Name', 'Middle_Name',
       'Primary_Specialty', 'Primary_Hospital_Affiliation', 'Last_Name',
       'Address1', 'SYM_ME_dhc'],
      dtype='object', length=221)

In [118]:
for col in merge_3.columns:
    print(col)

ME
RECORD_ID
UPDATE_TYPE
ADDRESS_TYPE
MAILING_NAME
LAST_NAME
FIRST_NAME
MIDDLE_NAME
SUFFIX
MAILING_LINE_1
MAILING_LINE_2
CITY
STATE
ZIP
SECTOR
CARRIER_ROUTE
ADDRESS_UNDELIVERABLE_FLAG
FIPS_COUNTY
FIPS_STATE
PRINTER_CONTROL_CODE
PC_ZIP
PC_SECTOR
DELIVERY_POINT_CODE
CHECK_DIGIT
PRINTER_CONTROL_CODE_2
REGION
DIVISION
GROUP
TRACT
SUFFIX_CENSUS
BLOCK_GROUP
MSA_POPULATION_SIZE
MICRO_METRO_IND
CBSA
CBSA_DIV_IND
MD_DO_CODE
BIRTH_YEAR
BIRTH_CITY
BIRTH_STATE
BIRTH_COUNTRY
GENDER
TELEPHONE_NUMBER
PRESUMED_DEAD_FLAG
FAX_NUMBER
TOP_CD
PE_CD
PRIM_SPEC_CD
SEC_SPEC_CD
MPA_CD
PRA_RECIPIENT
PRA_EXP_DT
GME_CONF_FLG
FROM_DT
TO_DT
YEAR_IN_PROGRAM
POST_GRADUATE_YEAR
GME_SPEC_1
GME_SPEC_2
TRAINING_TYPE
GME_INST_STATE
GME_INST_ID
MEDSCHOOL_STATE
MEDSCHOOL_ID
MEDSCHOOL_GRAD_YEAR
NO_CONTACT_IND
NO_WEB_FLAG
PDRP_FLAG
PDRP_START_DT
POLO_MAILING_LINE_1
POLO_MAILING_LINE_2
POLO_CITY
POLO_STATE
POLO_ZIP
POLO_SECTOR
POLO_CARRIER_ROUTE
MOST_RECENT_FORMER_LAST_NAME
MOST_RECENT_FORMER_MIDDLE_NAME
MOST_RECENT_FORMER_FIRS

In [121]:
merge_3 = merge_3.fillna('None')

In [130]:
THIS = merge_3[['ME','PHONE','PHONE_iqvia','Phone_Number']]

In [163]:
dict_list = []
for row in THIS.itertuples():
    count = 0
    phone_num = 'None'
    symph_phone = str(row.PHONE).replace('-','').replace(')','').replace('(','').replace(' ','')
    dhc_phone = str(row.Phone_Number).replace('.','')
    iqvia_phone = str(row.PHONE_iqvia).replace(' ','')
    print(iqvia_phone)
    if symph_phone == dhc_phone:
        MATCHES = 'Symphony, DHC'
        phone_num = symph_phone
        if dhc_phone == iqvia_phone:
            count = 2
            MATCHES = 'Symphony, DHC, IQVia'
        else:
            count = 1
    elif symph_phone == iqvia_phone:
        phone_num = symph_phone
        MATCHES = 'Symphony, IQVia'
        count = 1
    elif dhc_phone == iqvia_phone:
        phone_num = dhc_phone
        MATCHES = 'DHC, IQVia'
        count = 1
    if phone_num!='None':
        dicto = {
        'ME': row.ME,
        'PHONE': phone_num,
        'MATCHED': count,
        'MATCHES': MATCHES
    }
        dict_list.append(dicto)

2563864196
2567372000
7349364000
2059343411
7707512500
7707935000
3342724670
3342778330
3342882100
3343650651
3344184100
3346705000
3347493411
8507691511
9168437000
2057833000
6627724000
None
8508621111
7063786800
7065095000
7702282721
7704001000
None
5596242000
5596246011
5596246080
8084336661
6019841000
8504164000
8504167000
None
None
None
3122274000
2569734000
None
2052743000
2053383301
2058383000
2058385100
2567372000
5094557844
5094585800
None
None
7708242000
7708369666
None
None
8013504111
6507234000
3077464491
None
2516331000
2562651000
2563297100
6783121000
6783126800
5016867000
2053334500
2057597111
2057597241
5048423000
5048999311
None
2059338101
6153421000
7035585000
2144567000
None
7062726000
2059397000
4237786011
8037749000
8037912000
6153225000
9543554400
9543554400
3144546000
None
None
5034726131
None
2058771000
8505056601
None
6153225000
2059399100
2562651000
4047855437
None
6176677000
2516331000
2514352400
7759824100
9795436251
9852802200
8436921000
5072555123
50726678

9045427300
2075648401
2076955200
2079737000
None
None
9094275000
None
3122274000
5302256000
7759824100
7759825437
7759827000
9098257084
5032766500
5034132200
5034137711
5052651711
3103255111
None
5104283000
6195436222
8056526000
9168437000
4159257000
3033998020
None
9086852200
9739715000
2408266000
3018963100
None
9167844000
9169735000
None
2066241144
None
None
None
None
None
2083672121
5418817000
9164233000
9164534545
7074648511
4067517500
8015077000
2156624000
8593235000
9015457100
2098351500
None
None
9167844000
2094686000
5307566440
9167342011
5103071500
5107521000
5032161234
5416867300
5302256000
4088511000
None
6066797441
6066797441
None
None
5302256000
7076511000
8089323000
5303365511
None
None
2095781211
7853503111
8056526000
8059338600
None
5032161234
3077787550
7075764000
None
7072535000
None
9164233000
9164534545
9169837400
7073934000
5302746000
9164233000
9164534545
9169837400
9252954000
2143207000
9074528181
8708194040
5098653105
None
None
4078902273
7067330188
7603792681


None
3149968000
None
9167844000
None
9042440411
2102975000
2105754000
2105758110
2523998040
2523998998
4045011000
None
7033913600
7037764001
7037764002
5094585800
7122461230
8122751200
8123366821
9146810600
None
5034948311
8582665000
3104233277
5628268000
7144567890
3179445000
None
None
None
None
5025623000
6195025800
6195223600
8589393400
7035585000
None
8136815551
5034948311
None
None
None
2283776510
5034132200
5034137711
7067212273
3055851111
None
4434448000
4437777000
7863083000
7864672000
7865961960
7865961960
7866624000
None
7067212273
9109076000
2393546000
None
9034085000
None
None
3052435302
3053255511
3055851111
2063866000
4089472500
8178654400
8055435353
8055467600
2139896100
7148273000
None
5206940111
7079355000
6414724111
None
4173357000
3302638100
3308341111
None
None
None
9543443000
9545385000
9549418300
9549872000
None
None
2293331000
2294823110
2295437100
None
4078922135
3218415111
9419179000
None
None
6622931000
None
3218415111
4073032200
4073035600
4073038110
40764670

7737021000
5012571000
5012571000
5012571000
6177325500
5632445555
6308592222
8474375500
4348468437
5409497191
None
2317263511
6148984000
7658321234
8122387000
7603403911
2193261234
7082169000
3097795000
8129346624
None
None
8138447000
9287572101
5073732384
5074337351
9207293100
9207314101
9209697900
6163919000
6167747444
2172231200
5638752921
5638757101
9199664131
None
3122274000
5746471000
5746471000
5749484000
8473810123
3122274000
3129262000
3144546000
6186071000
7736653000
None
7043552000
7043812000
None
None
3179445000
3179445000
3179622000
2024442000
2028777000
3018688000
5049885263
2122637300
2122637300
2125986000
3018688000
None
6505732222
None
None
7012342000
2293331000
7576687000
6153274751
7709918000
2067443000
None
None
2623291000
None
6033565461
7853546000
7857763322
7165922871
7737021000
6082636400
8159684400
8159684400
2168441000
2168441262
2168443941
3173382345
None
None
2172459541
2172459541
4109555000
4109555089
None
None
6195436222
2146455555
None
None
4148053000
Non

9199664131
9199664131
9849740000
None
None
7022337000
None
None
6035772000
8137454673
2523998040
3175540000
2173833311
None
None
None
None
3172173000
3176216262
3173551411
3176216262
2123052500
8128825220
None
6163925141
None
None
4256885000
4142662000
3128646000
7657473111
None
2197385500
2198864000
None
3093438131
6065281212
7656492511
4047855437
4048518000
2708255100
2708255450
5025623000
5025874011
5026296000
5026298000
5026367111
5028978100
8129447701
7154213140
7154236060
7154241600
None
5043492581
5045926600
5047023000
5048977011
5048999511
5052722111
5052722111
None
None
8123794441
9804873000
9104504300
None
None
2696835510
8109875000
None
2483385000
2489645100
8508621111
4234952525
4234957100
4237783300
None
None
3175285000
3177454451
None
8646754000
8476953200
9204333500
None
None
None
7704344567
9124861000
9127646236
None
None
3173382345
3175827000
5202853000
8057373300
5016867000
5743355000
8159331671
2608243210
None
8585761700
4078462266
4078922135
3173382345
3175827000
76

...

2529754100
7244507095
7244585442
3042433000
8143712200
3052847500
8632931121
4062382500
None
7066478111
9517374343
None
None
None
None
None
6176365000
6176365000
4102258000
4103285887
4103288667
4103371000
5052722111
5052722111
8123794441
8653059000
8653059000
8188810800
5712313224
9733225000
None
5856714300
5859224000
7165683600
7168595600
None
4126472345
5707237764
7328402200
8434312000
None
None
5097585511
8285805000
None
None
None
5407434561
9143663000
9146930700
7407731141
9284452700
4344473151
9105772345
2523350531
2123125000
5184713221
5188421900
3026453300
8507698341
8185021900
4805156296
6023445011
6023445051
9705862317
8059882500
None
None
9287632273
5708886666
4706447000
7706441000
7707512500
7707935000
4807288000
6235845100
9035314080
9035938441
9036063000
5138621400
5614984440
None
3162832700
2157102000
6095995000
8709724100
8655411111
None
None
2159556000
3026514000
7149973000
9493641400
None
5629335437
5629335437
2168441000
2168443911
2168443941
2293121000
4407038922
508

4042565131
4048518000
6783121000
6788437001
2025746000
2025746000
8136815551
7067330188
None
2103584000
3182124000
4693227000
2164442200
7153871713
3054458461
8632931121
9036546800
5406891000
7242836666
7242844526
None
4846288000
6103788800
9372088000
9374382400
9377346784
9569942000
None
None
2105754000
2105756800
9783683000
None
7403565000
9372263200
4793388000
4795531000
3342882100
4042654000
2173422121
5025623000
6075473456
None
9417664122
None
None
None
9206223257
6192322946
None
3522533333
2536974000
None
5203934700
5209013500
8709361000
None
None
5135232111
5137129200
9375481141
2293121000
7709790200
None
2086254000
None
4349240000
None
9015165200
None
2483983200
7177826420
8144525000
5164966500
3179445000
3179622000
3174625544
3174776789
4192513232
6075473456
3192728000
7062726000
3097795000
3097795000
5632649100
5637425000
9167342011
8142749300
9046398500
7324312000
9733832121
7082169000
7082169000
3053255511
5173641000
7707935000
7323631900
7329423588
2102975000
5207921450
81

None
None
None
7404544000
None
5738849884
6184637311
9735381800
None
None
5104283000
None
9739724300
2164442200
6613825000
9374982311
7185901800
7708369666
None
6053121000
6053331000
6053331000
7012342000
8124505000
None
None
5738759000
9496422734
None
None
None
None
5086745600
5207921450
None
None
3043291400
3045981200
8552815728
4105354000
None
5083341000
7012342000
5852752100
5852757520
5854732200
2059307000
5419638421
6195436222
7012342000
4154442000
7186307000
None
9199664131
9849740000
8477232210
8477232210
6262564673
9099852811
6053228000
6053228000
6055041000
6059777000
7123622631
8433477111
7027318000
7027318000
9252954000
None
2075648401
2076955200
2079737000
None
None
3372892000
2036884242
5614228262
8604420711
8608898331
7182269000
8139722000
4134472000
None
8028470000
2164442200
7163232000
7163661111
8002232273
2036884242
6173556000
None
6177262000
None
6036683545
5594484500
5594503000
2253744325
2257656565
5043923131
5048977011
7015304894
7015307000
4052727000
None
None
N

7322225200
7322225200
8148892011
7342466000
2149478181
9167844000
None
5173647650
7032077100
7657473111
7657473281
7657515253
2146180500
6189422171
5703211000
6316898333
8145349000
6314746000
7067229011
9898393000
3179445000
3179445000
3179622000
None
3036894000
7207771234
None
6145448000
6145669000
3096475240
3096475240
3096475240
3096475240
7153871713
8669514321
None
None
6182424600
None
8433477111
8436921000
None
5617983300
5618426141
5619657300
9417926611
9547212200
9549740400
5757483333
None
None
None
5854732200
5854732200
8507630017
6603585700
9015238990
7076511000
None
6023445011
6014830011
6017748214
6017766925
7694861000
3135761000
None
None
2813322511
5803322323
9035776000
3034644500
7208480000
6073374111
None
3042630811
6013661712
6013761000
2024442000
2028777000
3017748882
3046233461
2813642300
9853452700
9199348171
8015812121
8015877000
4845264000
7328283000
None
3145775600
3145778000
3147688000
5738824141
5738824141
6369331000
6369331818
7733565000
2033740007
None
4697641

4059493011
6366255200
6367556500
9123679841
9123679841
2014181000
2253744325
2257656565
5736325000
5738158000
6182337750
6184637311
6369169000
None
6103771300
None
6262895454
6265701606
6265709000
6265732222
6265797777
6268988000
7025642622
9516882211
8328241000
8085389011
None
5733313000
None
None
3122274000
6362398000
4046055000
4047855437
4047855437
4047855437
4048518000
8328241000
8328263000
6262564673
3144546000
3149965000
3368842222
3368850141
6363441000
6369169000
None
5124472211
5629335437
None
None
None
None
None
9704957000
5036921212
None
3054458461
3214349000
2018333000
2019962000
3143446000
6369475000
6369475010
2707567000
7312212200
2069872000
5099464611
None
None
3216362211
3527518000
2106382000
None
9104573800
None
None
None
3052645252
7863083000
None
4046161000
6783126800
6147222000
4103285887
4103288667
4178834022
6087850940
5072555123
5072667890
5074337351
9283442000
2285235000
None
None
2149478181
9042440411
5802867623
2028777000
7134343800
9035318527
9035970351
6022

In [164]:
mess = pd.DataFrame(dict_list).drop_duplicates()

In [173]:
uhuh = merge_3[merge_3.ME.isin(mess.ME)][['ME','TELEPHONE_NUMBER','PHONE','PHONE_iqvia','Phone_Number','PHYSICAL_ADDR_1','ADDR_1','Address','ADDR_1_POLO','ADDR_1_PPMA']]

In [153]:
polos.columns

Index(['ME', 'RECORD_ID', 'UPDATE_TYPE', 'ADDRESS_TYPE', 'MAILING_NAME',
       'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'MAILING_LINE_1',
       ...
       'POST_CD_ID_PPMA', 'ADDR_1_PPMA', 'ADDR_2_PPMA', 'ADDR_3_PPMA',
       'CITY_PPMA', 'STATE_PPMA', 'ZIP_PPMA', 'FROM_DT_PPMA', 'POLO_NEWER',
       'SYM_ME'],
      dtype='object', length=101)

In [165]:
mess.groupby('MATCHES').count()

,ME,PHONE,MATCHED
MATCHES,,,
"DHC, IQVia",521,521,521
"Symphony, DHC",4779,4779,4779
"Symphony, DHC, IQVia",273,273,273
"Symphony, IQVia",901,901,901


In [170]:
iqvia_merge[['ME','AFFIL_RANK','AFFIL_TYPE_DESC']]

,ME,AFFIL_RANK,AFFIL_TYPE_DESC
0,00102000412,9,Admitting
1,00102000412,6,Admitting
2,00102011295,6,Admitting
3,00102011741,6,IDN Affiliated
4,00102020502,9,IDN Affiliated (Inferred)
...,...,...,...
26489,95708860060,8,IDN Affiliated (Inferred)
26490,95708880109,7,Admitting
26491,95708880133,10,Attending
26492,95708930033,8,Admitting


In [172]:
len(mess)

6474

In [177]:
one_day = pd.merge(uhuh, mess, on='ME', suffixes = ['_symphony','']).drop_duplicates()

In [189]:
one_day.drop_duplicates(['ME','PHONE'])

,ME,TELEPHONE_NUMBER,PHONE_symphony,PHONE_iqvia,Phone_Number,PHYSICAL_ADDR_1,ADDR_1,Address,ADDR_1_POLO,ADDR_1_PPMA,PHONE,MATCHED,MATCHES
0,00102020502,None,(770) 424-9732,7707512500,770.424.9732,3000 HOSPITAL BLVD,61 WHITCHER ST NE,2500 Hospital Blvd,2222 N Nevada Ave,210 Sherwood Pass,7704249732,1,"Symphony, DHC"
2,00102021126,2365341328.0,(334) 272-4670,3342724670,334.272.4670,215 PERRY HILL RD,215 PERRY HILL RD,215 Perry Hill Rd,527 N Palo Alto Ave,566 Lake Cameron Dr,3342724670,2,"Symphony, DHC, IQVia"
18,00102031164,None,(205) 206-8477,2057833000,205.206.8477,701 PRINCETON AVE SW,832 PRINCETON AVE SW,832 Princeton Ave Sw,832 Princeton Ave SW,7601 Southcrest Pkwy,2052068477,1,"Symphony, DHC"
20,00102060229,8282132325.0,(828) 213-2325,7063786800,828.213.2325,304 TURNER MCCALL BLVD SW,509 BILTMORE AVE,509 Biltmore Ave,509 Biltmore Ave,15 Belle Meade Dr SW,8282132325,1,"Symphony, DHC"
24,00102090144,5043451172.0,(662) 377-7170,None,662.377.7170,None,830 S GLOSTER ST,830 S Gloster St,830 S Gloster St,1758 Park Pl,6623777170,1,"Symphony, DHC"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10053,95708930033,None,(302) 994-2511,8037764000,302.994.2511,6439 GARNERS FERRY RD,1601 KIRKWOOD HWY,1601 Kirkwood Hwy,1601 Kirkwood Hwy,224 Autumn Cir,3029942511,1,"Symphony, DHC"
10054,96501860058,8165477037.0,(316) 284-6400,3162846400,316.284.6400,1901 E 1ST ST,1901 E 1ST ST,1901 E 1st St,1901 E 1st St,2651 Moolio Pl,3162846400,2,"Symphony, DHC, IQVia"
10055,00512031501,None,None,9098257084,909.825.7084,11201 BENTON ST,None,11201 Benton St,None,None,9098257084,1,"DHC, IQVia"
10056,03006151611,None,None,6177325500,617.732.5500,75 FRANCIS ST,None,75 Francis St,None,None,6177325500,1,"DHC, IQVia"


In [190]:
THIS_COUNT = 0
for row in one_day.drop_duplicates(['ME','PHONE']).itertuples():
    if str(row.TELEPHONE_NUMBER).replace('.0','') == row.PHONE:
        
        THIS_COUNT += 1

In [191]:
THIS_COUNT

1855

In [197]:
addresses =[]
for row in one_day.itertuples():
#     polo = str(row.ADDR_1_POLO).upper()
    iqvia = str(row.PHYSICAL_ADDR_1).upper()
#     ppma = str(row.ADDR_1_PPMA).upper()
    dhc = str(row.Address).upper()
    symphony = str(row.ADDR_1).upper()

    if check_match(symphony, dhc)==1:
        address = symphony
    elif check_match(symphony, iqvia)==1:
        address = symphony
    elif check_match(dhc, iqvia):
        address = iqvia
    else:
        address = 'None'
    addresses.append(address)

In [198]:
one_day['ADDRESS']=addresses

In [200]:
polo_matches =[]
ppma_matches =[]
phone_matches =[]
for row in one_day.itertuples():
    ppma = str(row.ADDR_1_PPMA).upper()
    polo = str(row.ADDR_1_POLO).upper()
    phone_2 = str(row.TELEPHONE_NUMBER).replace('.0','')
    polo_match = check_match(polo, row.ADDRESS)
    ppma_match = check_match(row.ADDRESS, ppma)
    phone_match = check_phone_match(row.PHONE, phone_2)
    polo_matches.append(polo_match)
    phone_matches.append(phone_match)
    ppma_matches.append(ppma_match)
        

In [ ]:
    if address != 'None':
        polo_match = check_match(polo, address)
        ppma_match = check_match(address, ppma)
        

In [204]:
one_day['PHONE_MATCH']=phone_matches
one_day['PPMA_MATCH']=ppma_matches
one_day['POLO_MATCH']=polo_matches

In [214]:
one_day[(one_day.MATCHED == 2)&(one_day.ADDRESS != 'None')]

,ME,TELEPHONE_NUMBER,PHONE_symphony,PHONE_iqvia,Phone_Number,PHYSICAL_ADDR_1,ADDR_1,Address,ADDR_1_POLO,ADDR_1_PPMA,PHONE,MATCHED,MATCHES,ADDRESS,PHONE_MATCH,PPMA_MATCH,POLO_MATCH
2,00102021126,2365341328.0,(334) 272-4670,3342724670,334.272.4670,215 PERRY HILL RD,215 PERRY HILL RD,215 Perry Hill Rd,527 N Palo Alto Ave,566 Lake Cameron Dr,3342724670,2,"Symphony, DHC, IQVia",215 PERRY HILL RD,0,0,0
4,00102021126,2365341328.0,(334) 272-4670,3342778330,334.272.4670,400 TAYLOR RD,215 PERRY HILL RD,215 Perry Hill Rd,527 N Palo Alto Ave,566 Lake Cameron Dr,3342724670,2,"Symphony, DHC, IQVia",215 PERRY HILL RD,0,0,0
6,00102021126,2365341328.0,(334) 272-4670,3342882100,334.272.4670,2105 E SOUTH BLVD,215 PERRY HILL RD,215 Perry Hill Rd,527 N Palo Alto Ave,566 Lake Cameron Dr,3342724670,2,"Symphony, DHC, IQVia",215 PERRY HILL RD,0,0,0
8,00102021126,2365341328.0,(334) 272-4670,3343650651,334.272.4670,124 S MEMORIAL DR,215 PERRY HILL RD,215 Perry Hill Rd,527 N Palo Alto Ave,566 Lake Cameron Dr,3342724670,2,"Symphony, DHC, IQVia",215 PERRY HILL RD,0,0,0
10,00102021126,2365341328.0,(334) 272-4670,3344184100,334.272.4670,1015 MEDICAL CENTER PKWY,215 PERRY HILL RD,215 Perry Hill Rd,527 N Palo Alto Ave,566 Lake Cameron Dr,3342724670,2,"Symphony, DHC, IQVia",215 PERRY HILL RD,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9890,91304860079,8034353250.0,(260) 426-5431,2604265431,260.426.5431,2121 LAKE AVE,2121 LAKE AVE,2121 Lake Ave,1854 Shackleford Ct,3833 Nanlyn Farm Cir,2604265431,2,"Symphony, DHC, IQVia",2121 LAKE AVE,0,0,0
9917,91316780097,7188366600.0,(718) 836-6600,7188366600,718.836.6600,800 POLY PL,22 KINGS RD,800 Poly Pl,800 Poly Pl,1400 S Ocean Dr,7188366600,2,"Symphony, DHC, IQVia",800 POLY PL,1,0,1
9926,91332030022,None,(715) 393-3000,7153933000,715.393.3000,3400 MINISTRY PKWY,3400 MINISTRY PKWY,3400 Ministry Pkwy,3400 Ministry Pkwy,3689 Oakwood Trl,7153933000,2,"Symphony, DHC, IQVia",3400 MINISTRY PKWY,0,0,1
10052,95708880133,None,(713) 791-1414,7137911414,713.791.1414,2002 HOLCOMBE BLVD,2002 HOLCOMBE BLVD,2002 Holcombe Blvd,323 E Chestnut St Rm 518,2002 Holcombe Blvd,7137911414,2,"Symphony, DHC, IQVia",2002 HOLCOMBE BLVD,0,1,0


In [217]:
one_day[(one_day.PPMA_MATCH == 0)&(one_day.PHONE_MATCH == 0)&(one_day.POLO_MATCH == 0)&(one_day.ADDRESS != 'None')].drop_duplicates('ME').to_csv('../../Data/POLO_Filter/Sample.csv', index=False)